## Task 2: Basic ML on MNIST and FashionMNIST

In [1]:
using MLDatasets, Plots, Measures, LaTeXStrings, StatsBase, Random, LinearAlgebra

In [ ]:
train_data = FashionMNIST.traindata(Float64);
train_imgs = train_data[1];
train_labels = train_data[2];

test_data = FashionMNIST.testdata(Float64);
test_imgs = test_data[1];
test_labels = test_data[2];
class_names = FashionMNIST.classnames()

In [ ]:
#training and testing images for FashionMNIST
tshirt_imgs, test_tshirt_imgs = train_imgs[:,:,train_labels .== 0], test_imgs[:,:,test_labels .== 0];
trousers_imgs, test_trousers_imgs = train_imgs[:,:,train_labels .== 1], test_imgs[:,:,test_labels .== 1];
pullover_imgs, test_pullover_imgs = train_imgs[:,:,train_labels .== 2], test_imgs[:,:,test_labels .== 2];
dress_imgs, test_dress_imgs = train_imgs[:,:,train_labels .== 3], test_imgs[:,:,test_labels .== 3];
coat_imgs, test_coat_imgs = train_imgs[:,:,train_labels .== 4], test_imgs[:,:,test_labels .== 4];
sandle_imgs, test_sandle_imgs = train_imgs[:,:,train_labels .== 5], test_imgs[:,:,test_labels .== 5];
shirt_imgs, test_shirt_imgs = train_imgs[:,:,train_labels .== 6], test_imgs[:,:,test_labels .== 6];
sneaker_imgs, test_sneaker_imgs = train_imgs[:,:,train_labels .== 7], test_imgs[:,:,test_labels .== 7];
bag_imgs, test_bag_imgs = train_imgs[:,:,train_labels .== 8], test_imgs[:,:,test_labels .== 8];
ankle_boot_imgs, test_ankle_boot_imgs = train_imgs[:,:,train_labels .== 9], test_imgs[:,:,test_labels .== 9];

In [ ]:
#only testing 1000 since my computer could plot the heatmap without crashing (out of memmory only 8gb sadge)
tshirt_imgs_as_vectors, test_tshirt_imgs_as_vectors = vcat([vec(tshirt_imgs[:,:,k])' for k in 1:1000]...), vcat([vec(test_tshirt_imgs[:,:,k])' for k in 1:1000]...);
trousers_imgs_as_vectors, test_trousers_imgs_as_vectors = vcat([vec(trousers_imgs[:,:,k])' for k in 1:1000]...), vcat([vec(test_trousers_imgs[:,:,k])' for k in 1:1000]...);
dress_imgs_as_vectors, test_dress_imgs_as_vectors = vcat([vec(pullover_imgs[:,:,k])' for k in 1:1000]...), vcat([vec(test_pullover_imgs[:,:,k])' for k in 1:1000]...); #last(size(pullover_imgs))
coat_imgs_as_vectors, test_coat_imgs_as_vectors = vcat([vec(dress_imgs[:,:,k])' for k in 1:1000]...), vcat([vec(test_dress_imgs[:,:,k])' for k in 1:1000]...); #last(size(dress_imgs))
sandle_imgs_as_vectors, test_sandle_imgs_as_vectors = vcat([vec(coat_imgs[:,:,k])' for k in 1:1000]...), vcat([vec(test_coat_imgs[:,:,k])' for k in 1:1000]...); #last(size(coat_imgs))
shirt_imgs_as_vectors, test_shirt_imgs_as_vectors = vcat([vec(sandle_imgs[:,:,k])' for k in 1:1000]...), vcat([vec(test_sandle_imgs[:,:,k])' for k in 1:1000]...); #last(size(sandal_imgs))
sneaker_imgs_as_vectors, test_sneaker_imgs_as_vectors = vcat([vec(shirt_imgs[:,:,k])' for k in 1:1000]...), vcat([vec(test_shirt_imgs[:,:,k])' for k in 1:1000]...); #last(size(shirt_imgs))
bag_imgs_as_vectors, test_bag_imgs_as_vectors = vcat([vec(bag_imgs[:,:,k])' for k in 1:1000]...), vcat([vec(test_sneaker_imgs[:,:,k])' for k in 1:1000]...); #last(size(bag_imgs))
ankle_boot_imgs_as_vectors, test_ankle_boot_imgs_as_vectors = vcat([vec(ankle_boot_imgs[:,:,k])' for k in 1:1000]...), vcat([vec(test_bag_imgs[:,:,k])' for k in 1:1000]...); #last(size(ankle_boot_imgs))

In [4]:
train_data_MINST = MLDatasets.MNIST.traindata(Float64);
train_imgs_MINST = train_data_MINST[1];
train_labels_MINST = train_data_MINST[2];

test_data_MINST = MLDatasets.MNIST.testdata(Float64);
test_imgs_MINST = test_data_MINST[1];
test_labels_MINST = test_data_MINST[2];

n_train_MINST, n_test_MINST = length(train_labels_MINST), length(test_labels_MINST)


X = vcat([vec(train_imgs_MINST[:,:,k])' for k in 1:n_train_MINST]...);
X_sorted = vcat([vec(train_imgs_MINST[:,:,k])' for k in sortperm(train_labels_MINST)]...);
#heatmap(X, legend=false)

┌ Warning: MNIST.traindata() is deprecated, use `MNIST(split=:train)[:]` instead.
└ @ MLDatasets C:\Users\joshu\.julia\packages\MLDatasets\soHS1\src\datasets\vision\mnist.jl:187
┌ Warning: MNIST.testdata() is deprecated, use `MNIST(split=:test)[:]` instead.
└ @ MLDatasets C:\Users\joshu\.julia\packages\MLDatasets\soHS1\src\datasets\vision\mnist.jl:195


In [5]:
A = [ones(n_train_MINST) X];
Adag = pinv(A);

In [6]:
using Flux: onehotbatch

tfPM(x) = x ? +1 : -1
yDat(k) = tfPM.(onehotbatch(train_labels_MINST,0:9)'[:,k+1])
bets = [Adag*yDat(k) for k in 0:9]; #this is the trained model (a list of 10 beta coeff vectors)


In [7]:
linear_classify(square_image) = argmax([([1 ; vec(square_image)])'*bets[k] for k in 1:10]) - 1

linear_classify (generic function with 1 method)

In [8]:
predictions = [linear_classify(test_imgs_MINST[:,:,k]) for k in 1:n_test_MINST]
confusionMatrix = [sum((predictions .== i) .& (test_labels_MINST .== j)) for i in 0:9, j in 0:9]
acc = sum(diag(confusionMatrix))/n_test_MINST

0.8603

### Task 2.1 One vs. all (rest) Linear and Logistic

In [9]:
include("src/ML.jl")

LoadError: LoadError: ArgumentError: Package FLUX not found in current path:
- Run `import Pkg; Pkg.add("FLUX")` to install the FLUX package.

in expression starting at c:\Users\joshu\Documents\MATH2504\Joshua-Crook-2504-2022-PROJECT3\src\ML.jl:4

ML.jl has the logistic training method.

In [ ]:
n_train, n_test = 10000, length(test_labels)

In [ ]:
A = [ones(n_train) train_data_all_classes];
Adag = pinv(A);

#### 1 vs all Logistic model for Fashion dataset
Determining if something is a t-shirt or not. 1 indicates tshirt 0 indicates it is not a t-shirt

In [ ]:
#creating one concatinated vector of all fashion data.
train_data_all_classes = vcat(tshirt_imgs_as_vectors, trousers_imgs_as_vectors, dress_imgs_as_vectors, coat_imgs_as_vectors, sandle_imgs_as_vectors, shirt_imgs_as_vectors, sneaker_imgs_as_vectors, bag_imgs_as_vectors, ankle_boot_imgs_as_vectors);

@show size(train_data_all_classes)
#heatmap(train_data_all_classes,legend=false)

In [ ]:

# firstly want to test if somthing is a tshirt or not.
train_labels_tshirt_vs_all = vcat(ones(1000),zeros(8000));

#shuffling data
Random.seed!(0)
perm = shuffle(1:9000)
train_data_tshirt_vs_all = train_data_all_classes[perm,:];
train_labels_tshirt_vs_all = train_labels_tshirt_vs_all[perm,:];
#heatmap(train_data_all_classes,legend=false)

In [ ]:
w1, b1 = train_logistic(train_data_tshirt_vs_all, train_labels_tshirt_vs_all, 9000);

Testing model

In [ ]:
test_non_tshirt_as_vector = vcat(test_trousers_imgs_as_vectors, test_dress_imgs_as_vectors, 
                                test_coat_imgs_as_vectors, test_sandle_imgs_as_vectors, 
                                test_shirt_imgs_as_vectors, test_sneaker_imgs_as_vectors, 
                                test_bag_imgs_as_vectors, test_ankle_boot_imgs_as_vectors);

In [ ]:
mean([logistic_classifier(test_tshirt_imgs_as_vectors',w1,b1) .== ones(1000)' logistic_classifier(test_non_tshirt_as_vector',w1,b1) .== zeros(8000)' ])

### Task 2.2 One vs. One Linear and Logistic

### Task 2.3 Multi-class classifier (logistic softmax)

### Task 2.4 Comparison of results and discussion